In [73]:
# pip install langchain-google-genai 
# pip install langchain

In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
google_gemini_api=os.getenv("GOOGLE_API_KEY")

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm_model=ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=google_gemini_api,verbose=False)

/Users/anuragsingh/Documents/GitHub/Mr-analyzer/analyzerenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1736657845.868457   20137 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1736657845.874189   20137 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [4]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [5]:
# TEMPLATE0: Generate a well-researched paragraph on a disease
TEMPLATE0="""
You are an expert in medicine, specializing in the human body and various medical fields that affect human health. Your extensive knowledge encompasses a wide range of diseases, treatments, and medications. 
Your task is to write a unique, 300-word paragraph on {disease_name} . Paragraph should be well-researched, informative, and distinct from previous ones, highlighting the causes, symptoms, treatments, and any other relevant information about the disease.
"""

In [6]:
para_generation_prompt = PromptTemplate(
    input_variables=["disease_name"],
    template=TEMPLATE0
)

In [7]:
para_chain=LLMChain(llm=llm_model, prompt=para_generation_prompt, output_key="para", verbose=False)

/var/folders/30/y95bh8d55nl10w9hr7kt73g00000gn/T/ipykernel_1762/3518450001.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  para_chain=LLMChain(llm=llm_model, prompt=para_generation_prompt, output_key="para", verbose=False)


In [8]:
TEMPLATE1="""
Text:{para}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for the disease {disease_name} for creating the public awareness. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["para", "number", "disease_name", "response_json"],
    template=TEMPLATE1
)

In [10]:
quiz_chain=LLMChain(llm=llm_model, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [11]:
TEMPLATE2="""
You are an medical expert and health advocates or public health advocates. Given a Multiple Choice Quiz for {disease_name} disease.\
You need to evaluate the quality of the question and give a complete analysis of the quiz. Only use at max 50 words for quality analysis. 
if the quiz is not at per with the quality and awreness and helpfull related to the public,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the public awareness and health
Quiz_MCQs:
{quiz}

Check from an medical expert and health advocates or public health advocates of the above quiz:
"""

In [12]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["disease_name", "quiz"], template=TEMPLATE2)

In [13]:
review_chain=LLMChain(llm=llm_model, prompt=quiz_evaluation_prompt, output_key="review")

In [14]:
generate_evaluate_chain=SequentialChain(
    chains=[para_chain,quiz_chain, review_chain], 
    input_variables=["disease_name", "number", "response_json"],
    output_variables=["para", "quiz", "review"], 
)

## loading the data for generating the MCQs

In [15]:
import pandas as pd
import json

In [16]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [17]:
MCQ_NUMBERS=5
DISEASE_NAME="Maleria"

In [18]:
response=generate_evaluate_chain(
    {
        "number":MCQ_NUMBERS,
        "disease_name":DISEASE_NAME,
        "response_json":json.dumps(RESPONSE_JSON)
    }
)

/var/folders/30/y95bh8d55nl10w9hr7kt73g00000gn/T/ipykernel_1762/3916710297.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new LLMChain chain...
Prompt after formatting:

Text:Malaria is a life-threatening parasitic disease transmitted by infected female Anopheles mosquitoes. It affects over 200 million people annually, predominantly in tropical and subtropical regions. The parasite, Plasmodium, invades red blood cells, causing symptoms that typically appear within 10-15 days after the bite. These symptoms include fever, chills, sweating, headache, muscle pain, and nausea. If left untreated, malaria can progress to severe complications such as cerebral malaria, respiratory distress, and organ failure.

Treatment for malaria involves antimalarial medications, such as artemisinin-based combination therapies (ACTs), which are highly effective in clearing the infection. Early diagnosis and treatment are crucial to prevent severe illness and death. Prevention measures include using insecticide-treated mosquito nets, insect repellents, and antimalarial prophylaxis for travelers visiting endemic area

In [19]:
response

{'number': 5,
 'disease_name': 'Maleria',
 'response_json': '{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}',
 'para': 'Malaria is a life-threatening parasitic disease transmitted by infected female Anopheles mosquitoes. It affects over 200 million people annually, predominantly in tropical and subtropical regions. The parasite, Plasmodium, invades red blood cells, causing symptoms that typically appear within 10-15 days after the bite. These symptoms include fever, chills, sweating, headache, muscle pain, and nausea. If left untreated, malaria can

In [93]:
new_quiz_df=pd.DataFrame([response])

In [94]:
new_quiz_df

,number,disease_name,response_json,para,quiz,review
0,5,Maleria,"{""1"": {""mcq"": ""multiple choice question"", ""opt...","Malaria, a deadly parasitic disease transmitte...","### RESPONSE_JSON\n{""1"": {""mcq"": ""Which of the...",**Quality Analysis:**\n\nThe quiz covers essen...


In [95]:
para_json=new_quiz_df.loc[0,'para'].strip()

In [96]:
para_json

'Malaria, a deadly parasitic disease transmitted by infected female Anopheles mosquitoes, poses a significant global health threat. The parasite, Plasmodium, enters the human body and invades red blood cells, causing a range of symptoms, including fever, chills, sweating, headache, and muscle pain. If left untreated, malaria can progress to severe complications, such as cerebral malaria, which can lead to coma and death. The disease is prevalent in tropical and subtropical regions, particularly in Africa, Southeast Asia, and Latin America, and disproportionately affects vulnerable populations, including pregnant women, children, and the immunocompromised. Treatment typically involves antimalarial medications, such as artemisinin-based combination therapies, which have significantly improved survival rates. However, resistance to antimalarial drugs remains a challenge, and ongoing research focuses on developing new and effective therapies. Preventive measures, including the use of insec

In [97]:
quiz_json = new_quiz_df.loc[0, 'quiz'].split('### RESPONSE_JSON\n')[1]
quiz_data = json.loads(quiz_json)

# Display the quiz questions
for key, value in quiz_data.items():
    print(f"Question {key}: {value['mcq']}")
    for option_key, option_value in value['options'].items():
        print(f"  {option_key}: {option_value}")
    print(f"Correct Answer: {value['correct']}\n")

Question 1: Which of the following is NOT a symptom of malaria?
  a: Fever
  b: Vomiting
  c: Headache
  d: Sweating
Correct Answer: b

Question 2: What is the primary mode of transmission for malaria?
  a: Contact with infected blood
  b: Mosquito bites
  c: Contaminated water
  d: Inhalation of spores
Correct Answer: b

Question 3: Which of the following is a preventive measure against malaria?
  a: Use of mosquito nets
  b: Vaccination
  c: Taking antibiotics
  d: None of the above
Correct Answer: a

Question 4: What is the most prevalent type of malaria parasite?
  a: Plasmodium falciparum
  b: Plasmodium vivax
  c: Plasmodium malariae
  d: Plasmodium ovale
Correct Answer: a

Question 5: Which region of the world is most affected by malaria?
  a: North America
  b: Europe
  c: Tropical and subtropical regions
  d: Antarctica
Correct Answer: c



In [98]:
quiz_list = []
for key, value in quiz_data.items():
    question = value['mcq']
    options = value['options']
    correct_ans = value['correct']
    row = {
        'question': question,
        'optiona': options.get('a', ''),
        'optionb': options.get('b', ''),
        'optionc': options.get('c', ''),
        'optiond': options.get('d', ''),
        'correctans': correct_ans
    }
    quiz_list.append(row)

# Convert to DataFrame
df_quiz = pd.DataFrame(quiz_list)

# Save to CSV
df_quiz.to_csv('quiz_questions.csv', index=False)